# Compute age-specific baselines for the US data using the Karlinsky & Kobak, 2021 model
Adapted from https://github.com/dkobak/excess-mortality/blob/main/baselines-stmf.ipynb

In [1]:
import numpy as np, pandas as pd
from sklearn.linear_model import LinearRegression

df_stmf = pd.read_csv("data/tidy_deaths_age_state_CDC.csv"
                   ).rename(columns = {"Age Group":"Age"})

df_stmf.head()

,Jurisdiction,Year,Age,Week,Number of Deaths
0,Alabama,2015,<65,1,345
1,Alabama,2015,<65,2,333
2,Alabama,2015,<65,3,303
3,Alabama,2015,<65,4,294
4,Alabama,2015,<65,5,316


In [2]:
years_with_53_weeks = [2009, 2015, 2020, 2026]

def predict(X):    
    # Fit regression model on pre-2020 data from 2015 on 
    ind = (X[:,0] >= 2015) & (X[:,0] < 2020) & (X[:,1]<53)

    m = int(np.max(X[ind,1]))
    onehot = np.zeros((np.sum(ind), m))
    for i,k in enumerate(X[ind,1]):
        onehot[i,int(k)-1] = 1
    predictors = np.concatenate((X[ind,:1], onehot), axis=1)
    reg = LinearRegression(fit_intercept=False).fit(predictors, X[ind,2])
        
    # Compute 2020 baseline
    ind2 = X[:,0] == 2020
    predictors2020 = np.concatenate((np.ones((m,1))*2020, np.eye(m)), axis=1)
    baseline = reg.predict(predictors2020)
    
    # Week 53 usually does not have enough data, so we'll use 
    # the same baseline value as for week 52
    baseline = np.concatenate((baseline, [baseline[-1]]))
    
    df = pd.DataFrame()
    
    for Year in range(2015, 2023):
        
        Week = [i for i in range(1, 53)]
        
        predictors_year = np.concatenate((np.ones((m,1))*Year, np.eye(m)), axis=1)
        baseline_year = reg.predict(predictors_year)
        
        df_tmp = pd.DataFrame(data = baseline_year, 
                              index = Week, 
                              columns = ["Number of Deaths"]) 
        df_tmp["Year"] = Year
        df_tmp = df_tmp.reset_index().rename(columns = {"index":"Week"})
        
        if Year in years_with_53_weeks:
            # Week 53 again
            last_row = df_tmp.iloc[-1].copy()
            last_row[0] = 53
            df_tmp = df_tmp.append(last_row)
            
        df = df.append(df_tmp)
            
    return df

In [3]:
countries = np.unique(df_stmf['Jurisdiction'])
agebands = ["<65", "65-74", "75-84", "85+"]
            
df = pd.DataFrame()

for country in countries:
    print('.', end='')
    for ageband in agebands:
        X = df_stmf[(df_stmf['Jurisdiction']==country) & (df_stmf['Age']==ageband)]
        X = X[['Year','Week', 'Number of Deaths']].values
        baselines  = predict(X)
        baselines["Jurisdiction"] = country
        baselines["Age Group"] = ageband
        df = df.append(baselines)
df["Type"] = "Predicted"
order = ["Jurisdiction", "Year", "Age Group", "Week", "Number of Deaths", "Type"]
df[order].set_index("Jurisdiction").to_csv("data/tidy_predicted_deaths_age_state_CDC.csv")

df.head()

.....................................................

,Week,Number of Deaths,Year,Jurisdiction,Age Group,Type
0,1.0,329.123077,2015.0,Alabama,<65,Predicted
1,2.0,333.923077,2015.0,Alabama,<65,Predicted
2,3.0,310.923077,2015.0,Alabama,<65,Predicted
3,4.0,312.123077,2015.0,Alabama,<65,Predicted
4,5.0,321.523077,2015.0,Alabama,<65,Predicted
